# Visualizing a .tif/.tiff file

This will be a short tutorial showing you how to code a program to view a .tif/.tiff file

In [1]:
import os, sys
from osgeo import ogr, osr, gdal
from ipyleaflet import Map, ImageOverlay, TileLayer, LayersControl, WMSLayer
import ipywidgets as ipyw
from notebook import notebookapp

In [2]:
# create a dictionary that will contain all the layers along with the file path to the image they display
layer_names = {}

# single-banded raster
# tmp_file_path = os.path.abspath('./BearCk.tif')

# multi-banded raster. When viewing this, be sure to wait and select the checkboxes when they appear
tmp_file_path = os.path.abspath('./canaryislands_tmo_2013166_geo.tif')

file_name = tmp_file_path.split('/')[-1].split('.tif')[0]
tile_folder_path = os.path.dirname(tmp_file_path) + '/'

In [3]:
# opens the file up to read
raster = gdal.Open(tmp_file_path)

# precautionary measure to ensure that the file that was uploaded was in the correct projection
# raster = gdal.Warp('./' + file_name + '_temp.tif', tmp_file_path, dstSRS="EPSG:4326", resampleAlg='bilinear')

# extracts the extents of the image from the file
file_path = os.path.abspath('./' + file_name + '_temp.tif')
x_min = raster.GetGeoTransform()[0]
x_max = x_min + raster.GetGeoTransform()[1]*raster.RasterXSize
y_max = raster.GetGeoTransform()[3]
y_min = y_max - abs(raster.GetGeoTransform()[5])*raster.RasterYSize

# sometimes the tiler ends up creating a folder that shouldn't be created when generating tiles for a geotiff that covers
# the entire map. The folder that ends up being created is a negative zoom level. The tiler program automatically defaults
# to generating zoom levels 0-3 for such a file anyway so this is just insurance in case there was the possibility of a
# negative zoom level
min_zoom_level=0
max_zoom_level=0
if x_max - x_min > 180 or y_max - y_min > 90:
    min_zoom_level=0
    max_zoom_level=3
        
# stores all the bands contained in the raster into a dictionary and associates a checkbox widget with each band
# this allows us to select which bands of the raster we want to put on the map
bands_dict = {}
if raster.RasterCount > 1: # when the raster has more than 1 band
    options = [];
    for band in range( raster.RasterCount ):
        band += 1
        bands_dict[str(band)] = ipyw.Checkbox(description=str(band), value=False)
        options.append(bands_dict[str(band)])
    options_widget = ipyw.VBox(options, layout={'overflow': 'scroll'})
    multi_select = ipyw.VBox([options_widget])
    display('select which bands you want to visualize')
    display(multi_select)
else: # when the raster has only one band
    bands_dict['1'] = 1

'select which bands you want to visualize'

In [4]:
%%time

# gets all the checked bands
selected_bands = []
if len(bands_dict) == 1:
    selected_bands = [1]
else:
    selected_bands = [int(k) for k, v in bands_dict.items() if v.value]


# iterates through all the checked bands and creates directories for each band to store the tiles. creates the tiles using the zoom levels generated from above
zoom = 0
for band in selected_bands:
    layer_names[file_name + '_band_' + str(band)] = file_path + ";" + tile_folder_path + file_name + '_band_' + str(band)
    if not os.path.exists(tile_folder_path + file_name + '_band_' + str(band)):
        os.mkdir(tile_folder_path + file_name + '_band_' + str(band))
    gdal.Translate('./output' + str(band) + '.vrt', file_path, bandList=[band], outputSRS='EPSG:4326')
    if not os.path.exists('./output'):
        gdal.Translate('./output' + str(band) + '.vrt', file_path, bandList=[band], outputSRS='EPSG:4326', rgbExpand='rgba') 
    
    if not min_zoom_level == max_zoom_level: 
        cmd = 'gdal2tiles.py -e -z ' + str(min_zoom_level) + '-' + str(max_zoom_level) + ' -a 0,0,0 ./output' + str(band) + '.vrt ' + tile_folder_path + file_name + '_band_' + str(band)
    else:
        cmd = 'gdal2tiles.py -e -a 0,0,0 ./output' + str(band) + '.vrt ' + tile_folder_path + file_name + '_band_' + str(band)

    if os.system(cmd) == 0:
        display(file_name + '_band_' + str(band) + ' tiles were successfully created')
    else:
        display('failure on creating ' + file_name + '_band_' + str(band) + ' tiles')
    
    os.remove('./output' + str(band) + '.vrt')
    
    for root, directory, files in os.walk(tile_folder_path + file_name + '_band_' + str(band)):
        zoom = directory[-1]
        break

Wall time: 0 ns


In [5]:
# intermediary in generating the url to get where the tiles are stored
base_url = "https://proxy.mygeohub.org"
nb = None
session = os.environ['SESSION']
servers = list(notebookapp.list_running_servers())
for server in servers:
    if session in server['base_url']:
        nb = server['base_url']
        nb_dir = server['notebook_dir']
        break

cwd = os.getcwd()
if 'notebooks/' in cwd: 
    bin_path = '/' + cwd.split('notebooks/')[1]
    if 'notebooks' not in nb_dir:
        bin_path = '/notebooks' + bin_path
else:
    bin_path = ''

KeyError: 'SESSION'

In [9]:
# creates an ipyleaflet map with the center at the origin of the file
m = Map(center = [(y_max + y_min)/2, (x_min + x_max)/2],
        zoom = int(zoom),
        scroll_wheel_zoom = True)
#         scroll_wheel_zoom = True,
#         crs = 'EPSG4326')
layer_cont = LayersControl()
m.add_control(layer_cont)

# iterates through all the layers added and creates tile layers for them. adds each tile layer to the map
# for k in layer_names:
#     print(base_url, nb, tile_folder_path)
#     url = base_url + nb + 'tree' + tile_folder_path.split(os.path.expanduser('~'))[-1] + k + '/{z}/{x}/{-y}.png'
#     test_layer = TileLayer(url = url, opacity=.75, name=k)
#     m.add_layer(test_layer)
print(tile_folder_path)
# url =  tile_folder_path.split(os.path.expanduser('~'))[-1] + k + '/{z}/{x}/{-y}.png'
url = "http://localhost:8889//files//data//geotiff_tutorial/canaryislands_tmo_2013166_geo.tif"
print(url)
test_layer = TileLayer(url = url, opacity=.75, name="canaryislands_tmo_2013166_geo.tif")
m.add_layer(test_layer)
display(m)

C:\Users\muhdr\Documents\simple-us\data\geotiff_tutorial/
http://localhost:8889//files//data//geotiff_tutorial/canaryislands_tmo_2013166_geo.tif


Map(center=[27.79909851935055, -15.998750590811774], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [16]:
# creates an ipywidget text area which will be where the coordinate and raster information will be stored. 
# creates a handler for mouse clicks which returns the coordinates and also prints the value of the raster
# on each band to the ipywidget text area
box = ipyw.Textarea(layout=ipyw.Layout(width='100%', height='100px'))
temp = []
def mouse_event(**kwargs):
    if kwargs.get('type') == 'click':
        message = 'coordinates = ' + str(kwargs.get('coordinates')) + '\n\n'
        temp.clear()
        temp.append(kwargs.get('coordinates'))
        for i in range(len(m.layers)):
            if not m.layers[i].name == '':
                value = os.popen("gdallocationinfo -valonly -b " + m.layers[i].name.split('_band_')[-1] 
                                 + " -wgs84 " + layer_names[m.layers[i].name].split(';')[0]
                                 + " " + str(temp[0][1]) + " " + str(temp[0][0])).read()
                message += m.layers[i].name + ' value = ' + value
        box.value = message
m.on_interaction(mouse_event)

NameError: name 'm' is not defined

In [17]:
display(box)
display(m)

Textarea(value='', layout=Layout(height='100px', width='100%'))

NameError: name 'm' is not defined

In [ ]:
# for visualizing a geotiff on your computer, try using this to upload the file onto jupyter notebook


# from hublib.ui import FileUpload

# def done(w, files):
#     display("{} was uploaded".format(files))

# # uploads the file to the server

# f = FileUpload("Select .tif or .tiff file",
#                "Upload File",
#                cb = done,
#                maxsize = '500M'
#               )
# f

# and then use this to get the tmp_file_path, file_name, and tile_folder_path

# tmp_file_path = os.path.abspath(f.list()[0])
# file_name = f.list()[0].split('/')[-1].split('.tif')[0]
# tile_folder_path = os.path.dirname(tmp_file_path) + '/'